### Model training

This script attempts to train a simple classification model using the tensorflow version of the quantization example repo https://github.com/sanchit88/tf_model_quant/tree/master.



In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directories for training and testing datasets
train_dir = r'C:\Peter\projects\2024-24-TAN\imgs\training-data-sets\current-train-set-SMALL\train'
test_dir = r'C:\Peter\projects\2024-24-TAN\imgs\training-data-sets\current-train-set-SMALL\test'

# Create ImageDataGenerators for training and testing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # MobileNetV2 requires 224x224 input size
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


In [3]:

# Load pre-trained MobileNetV2 without the top (classification) layer
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
base_model.trainable = False

# Add custom layers on top of MobileNetV2
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')  # Adjust to the number of classes
])


In [ ]:

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

# Save the model
model.save(r'.models/mobilenet-hippo.h5')
